In [64]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os
import random

In [65]:
def save_to_csv(df, filename):
    if not os.path.isfile(filename):
        df.to_csv(filename, index=False)
    else:  
        df.to_csv(filename, mode='a', header=False, index=False)

In [66]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [67]:
nhl_teams = ["ANA", "ARI", "BOS", "BUF", "CAR", "CBJ", "CGY", "CHI", "COL", "DAL", "DET", "EDM", "FLA", "LAK", "MIN", "MTL", "NJD",
             "NSH", "NYI", "NYR", "OTT", "PHI", "PIT", "SEA", "SJS", "STL", "TBL", "TOR", "VAN", "VGK", "WPG", "WSH"]

full_df = pd.DataFrame()

for team in nhl_teams:
    driver = webdriver.Chrome(options=options)
    url = f"https://edge.nhl.com/en/team/{team}"
    driver.get(url)
    time.sleep(10)

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    time.sleep(10)

    overview_div = soup.find('div', {'data-name': 'sportdata-profile-element', 'data-section': 'overview'})
    table_div = overview_div.find('div', class_='table-responsive')
    table = table_div.find('table')
    time.sleep(10)

    rows = table.find_all('tr')
    headers = [header.get_text(strip=True) for header in rows[0].find_all('td')]
    time.sleep(10)

    table_data = []
    for row in rows[1:]:
        cols = row.find_all(['td', 'th'])
        cols = [col.get_text(strip=True) for col in cols]
        table_data.append(cols)
        time.sleep(10)

    dataframe_headers = ['Stat', 'Team Score', 'League average', 'Percentile']
    df = pd.DataFrame(table_data, columns=dataframe_headers)
    df['Team'] = team
    full_df = pd.concat([full_df, df], ignore_index=True)
    
    driver.quit()
    time.sleep(random.randint(30, 40))

In [68]:
full_df.head(20)

,Stat,Team Score,League average,Percentile,Team
0,Top Skating Speed (mph),23.49,23.60,Below 50th,ANA
1,Speed Bursts Over 20 mph,"1,426","1,692",Below 50th,ANA
2,Skating Distance (mi),"3,616.74","3,703.23",Below 50th,ANA
3,Top Shot Speed (mph),101.69,100.38,81,ANA
4,Shots on Goal,"2,194","2,483",Below 50th,ANA
5,Shooting %,9.3%,10.2%,Below 50th,ANA
6,Goals,203,252,Below 50th,ANA
7,Off. Zone Time (ES),38.8%,40.8%,Below 50th,ANA
8,Top Skating Speed (mph),23.56,23.60,50,ARI
9,Speed Bursts Over 20 mph,"1,522","1,692",Below 50th,ARI


In [69]:
hockey_edge_data = r"C:\Users\gabri\Documents\hockey_edge_dataset.csv"
save_to_csv(full_df, hockey_edge_data)